# Variables

## Import des librairies

In [ ]:
import os
import sys
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

## Définition des variables globales

In [ ]:
# Fichiers existants
addr_ont_file_name = "address_ont.ttl"
ev_ont_file_name = "events_ont.ttl"
ont_file_name = "merge_addr_ont.ttl" # Fusion des deux fichiers précédents
# ruleset_file_name = "rules.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"

# Dossiers existants
data_folder_name = "data"
mapping_folder_name = "mappings"

# Dossiers créés durant le traitement
tmp_folder_name = "tmp_files"

project_name = "addresses_from_factoids"
graphdb_url = "http://localhost:7200"
ontorefine_url = "http://localhost:7333"
local_uri = "http://rdf.geohistoricaldata.org/id/address/"

# Commande pour lancer `Ontorefine`, dépend de l'OS et d'où il se situe
# ontorefine_cmd = "ontorefine-cli" # Nom sans chemin
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli" #Ubuntu
# ontorefine_cmd = "/Applications/Ontotext\ Refine.app/Contents/app/bin/ontorefine-cli" #MacOS

wd_graph_name = "wikidata"
addr_graph_name = "addresses_from_factoids"

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
addr_ont_file = os.path.abspath(addr_ont_file_name)
ev_ont_file = os.path.abspath(ev_ont_file_name)
ont_file = os.path.abspath(ont_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import wikidata as wd
import ontorefine as otr
import graphdb as gd
import graphrdf as gr
import strprocessing as sp

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [ ]:
# gd.remove_repository(graphdb_url, project_name)
gd.create_config_local_repository_file(local_config_file, project_name)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

### Vidage du répertoire local
Le répertoire dont l'id est `project_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [ ]:
gd.clear_repository(graphdb_url, project_name)

## Import des ontologies

In [ ]:
# gd.import_ttl_file_in_graphdb(graphdb_url, project_name, addr_ont_file)
# gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ev_ont_file)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ont_file)

# Import des sources

## Voies de Paris via Wikidata

### Variables liées à la source Wikidata

In [ ]:
# `wdpt` pour "wikidata paris thoroughfares"

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpt_csv_file_name = "wd_paris_thoroughfares.csv"
wdpt_csv_file = os.path.join(tmp_folder, wdpt_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpt_mapping_file_name = "mapping_voies_wikidata.json"
wdpt_mapping_file = os.path.join(mapping_folder, wdpt_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdpt_kg_file_name = "wd_paris_thoroughfares.ttl"
wdpt_kg_file = os.path.join(tmp_folder, wdpt_kg_file_name)

# Définition des graphes nommés
wdpt_graph_name = "wikidata"
wdpt_ids_graph_name = "wikidata_ids"
wdpt_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, wdpt_graph_name)
wdpt_ids_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, wdpt_ids_graph_name)

### Sélection des voies de Paris sur Wikidata via une requête de sélection

Via une requête SPARQL de sélection, on récupère une liste de ressources à créér :
* pour chaque ressource décrivant une voie dans Paris, on créé autant de ressources qu'elle a de nom officiel. La place de la Nation ([wd:Q1573359](https://www.wikidata.org/entity/Q1573359)) a quatre noms officiels donc on aura 4 ressources. Pour celles qui n'ont pas de nom officiel, on créé une seule ressource dont le nom est le label en français
* ces ressources créées seront liées à celle de Wikidata dont elle provient via `skos:closeMatch`
* chaque ressource créée suit la structure définie par l'ontologie
* le résultat de la requête est stocké dans un fichier csv qui sera converti en graphe de connaissance via Ontotext Refine et un fichier de mapping 

In [ ]:
query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
PREFIX source: <http://rdf.geohistoricaldata.org/id/address/sources/wikidata/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
PREFIX wb: <http://wikiba.se/ontology#>
PREFIX time: <http://www.w3.org/2006/time#>

SELECT ?landmarkId ?voie ?nomOff ?dateStartStamp ?dateStartPrec ?dateStartCal ?dateEndStamp ?dateEndPrec ?dateEndCal
WHERE {{

    BIND(CONCAT("LM_", STRUUID()) AS ?landmarkId)
    {{
        SELECT DISTINCT * WHERE {{
            {{?voie p:P361 [ps:P361 wd:Q16024163].}}UNION{{?voie p:P361 [ps:P361 wd:Q107311481].}}
            {{?voie p:P1448 ?nomOffSt.
                ?nomOffSt ps:P1448 ?nomOff.
                OPTIONAL {{?nomOffSt pq:P580 ?dateStartStamp; pqv:P580 [wb:timePrecision ?dateStartPrecRaw; wb:timeCalendarModel ?dateStartCal]}}
                OPTIONAL {{?nomOffSt pq:P582 ?dateEndStamp; pqv:P582 [wb:timePrecision ?dateEndPrecRaw; wb:timeCalendarModel ?dateEndCal]}}
            }}UNION{{
                ?voie rdfs:label ?nomOff.
                FILTER (LANG(?nomOff) = "fr")
                MINUS {{?voie p:P1448 ?nomOffSt}}
            }}
        }}
    }}
    
    BIND(IF(?dateStartPrecRaw = 11, time:unitDay, 
                     IF(?dateStartPrecRaw = 10, time:unitMonth,
                         IF(?dateStartPrecRaw = 9, time:unitYear,
                             IF(?dateStartPrecRaw = 8, time:unitDecade,
                                 IF(?dateStartPrecRaw = 7, time:unitCentury,
                                     IF(?dateStartPrecRaw = 6, time:unitMillenium, ?x
                                 )))))) AS ?dateStartPrec)
     BIND(IF(?dateEndPrecRaw = 11, time:unitDay, 
                     IF(?dateEndPrecRaw = 10, time:unitMonth,
                         IF(?dateEndPrecRaw = 9, time:unitYear,
                             IF(?dateEndPrecRaw = 8, time:unitDecade,
                                 IF(?dateEndPrecRaw = 7, time:unitCentury,
                                     IF(?dateEndPrecRaw = 6, time:unitMillenium, ?x
                                 )))))) AS ?dateEndPrec)
}}
"""

# wd.save_select_query_as_csv_file(query, wdpt_csv_file)

### Structuration des résultats en graphe de connaissances

In [ ]:
# Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
otr.get_export_file_from_ontorefine(wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file, ontorefine_cmd, ontorefine_url, project_name)

# Importer le fichier `wdpt_kg_file` qui a été créé lors de la ligne précédente dans le répertoire `project_name`, dans le graphe nommé `wdpt_graph_name` 
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, wdpt_kg_file, wdpt_graph_name)

### Transfert des liens `skos:closMatch` vers un autre graphe nommé

In [ ]:
query = f"""
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
DELETE {{
    GRAPH <{wdpt_graph_uri}> {{?s skos:closeMatch ?o}}
}}
INSERT {{
    GRAPH <{wdpt_ids_graph_uri}> {{?s skos:closeMatch ?o}}
}}
WHERE {{
    GRAPH <{wdpt_graph_uri}> {{?s skos:closeMatch ?o}}
}}
"""

gd.update_query(query, graphdb_url, project_name)

### Ajout du lien repère / source

In [ ]:
query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

INSERT DATA {{
    GRAPH <{wdpt_graph_uri}> {{
        facts:WD_Source a rico:Record;
            rdfs:label "Wikidata"@fr.
    }}
}} ;
INSERT {{
    GRAPH <{wdpt_graph_uri}> {{
        ?landmark rico:isOrWasDescribedBy facts:WD_Source.
    }}
}}WHERE {{
    GRAPH <{wdpt_graph_uri}> {{
        ?landmark a addr:Landmark.
        BIND(facts:WD_Source AS ?source)
    }}
}}
"""

gd.update_query(query, graphdb_url, project_name)

## Voies de Paris via les données de la ville de Paris

Import des données de Paris via les données de la ville de Paris :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Le travail se fait sur deux fichiers csv issus des deux jeux de données.

Chaque ligne des deux fichiers constitue une ressource dans le répertoire. Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

### Variables liées aux données de la ville de Paris

In [ ]:
# `vpt` pour "ville paris thoroughfares"

# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
vpta_mapping_file_name = "mapping_voies_paris_actuelles.json"
vpta_mapping_file = os.path.join(mapping_folder, vpta_mapping_file_name)
vptc_mapping_file_name = "mapping_voies_paris_caduques.json"
vptc_mapping_file = os.path.join(mapping_folder, vptc_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpta_kg_file_name = "voies_paris_actuelles.ttl"
vpta_kg_file = os.path.join(tmp_folder, vpta_kg_file_name)
vptc_kg_file_name = "voies_paris_caduques.ttl"
vptc_kg_file = os.path.join(tmp_folder, vptc_kg_file_name)

# Définition des graphes nommés
vpt_graph_name = "ville_de_paris"
vpt_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, vpt_graph_name)

### Structuration des résultats en graphe de connaissances

In [ ]:
# Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
otr.get_export_file_from_ontorefine(vpta_csv_file, vpta_mapping_file, vpta_kg_file, ontorefine_cmd, ontorefine_url, project_name)
otr.get_export_file_from_ontorefine(vptc_csv_file, vptc_mapping_file, vptc_kg_file, ontorefine_cmd, ontorefine_url, project_name)

# Importer le fichier `wdpt_kg_file` qui a été créé lors de la ligne précédente dans le répertoire `project_name`, dans le graphe nommé `wdpt_graph_name` 
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, vpta_kg_file, vpt_graph_name)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, vptc_kg_file, vpt_graph_name)

### Suppresion des instants temporels sans timeStamp

In [ ]:
query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
DELETE {{
    ?timeInstant ?p ?o.
    ?s ?p ?timeInstant.
}}WHERE {{
    GRAPH <{vpt_graph_uri}> {{
        ?timeInstant a addr:TimeInstant.
        MINUS {{?timeInstant addr:timeStamp ?timeStamp}}
        {{?timeInstant ?p ?o}}UNION{{?s ?p ?timeInstant}}
    }}
}}
"""

gd.update_query(query, graphdb_url, project_name)

### Ajout du lien repère / source

In [ ]:
query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>

INSERT DATA {{
    GRAPH <{vpt_graph_uri}> {{
        facts:DVP_Source a rico:Record;
            rdfs:label "dénomination des voies de Paris (actuelles et caduques)"@fr;
            rico:hasPublisher addr:DirTopoDocFoncVP .
        facts:DirTopoDocFoncVP a rico:CorporateBody;
            rdfs:label "Département de la Topographie et de la Documentation Foncière de la Ville de Paris"@fr.    
    }}
}} ;
INSERT {{
    GRAPH <{vpt_graph_uri}> {{
        ?landmark rico:isOrWasDescribedBy facts:DVP_Source.
    }}
}}WHERE {{
    GRAPH <{vpt_graph_uri}> {{
        ?landmark a addr:Landmark.
        BIND(facts:DVP_Source AS ?source)
    }}
}}
"""

gd.update_query(query, graphdb_url, project_name)

## Adresses de Paris à partir de la BAN

Import des adresses de Paris venant de la [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale), accessible [ici](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

### Variables liées aux données de la BAN

In [ ]:
# `bpa` pour "BAN paris addresses"

# Fichier CSV pour stocker le résultat de la requête de la sélection
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
bpa_mapping_file_name = "mapping_ban_adresses.json"
bpa_mapping_file = os.path.join(mapping_folder, bpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

# Définition des graphes nommés
ban_graph_name = "ban"
ban_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, ban_graph_name)

### Structuration des résultats en graphe de connaissances

In [ ]:
# Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
otr.get_export_file_from_ontorefine(bpa_csv_file, bpa_mapping_file, bpa_kg_file, ontorefine_cmd, ontorefine_url, project_name)

# Importer le fichier `wdpt_kg_file` qui a été créé lors de la ligne précédente dans le répertoire `project_name`, dans le graphe nommé `bpa_graph_name` 
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, bpa_kg_file, ban_graph_name)

### Nettoyage du graphe

Détection de repères décrits via plusieurs ressources et création d'une unique ressource pour chaque.

In [ ]:
# Détection des communes et arrondissements dupliqués et création d'une source centrale (?newLandmark) selon le code INSEE.

query1 = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geofla: <http://data.ign.fr/def/geofla#>
prefix sys: <http://www.ontotext.com/owlim/system#>
PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>

INSERT {
    GRAPH ?g {
        ?landmark skos:exactMatch ?tmpLandmark.
    }
}
WHERE
{
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?g)
    {
        SELECT DISTINCT ?insee {
            GRAPH ?g {
                ?tmpLandmark a addr:Landmark; geofla:numInsee ?insee.
            }
        }
    }
    BIND(URI(CONCAT(STR(URI(ban:)), "LM_", STRUUID())) AS ?landmark)

    GRAPH ?g {
        ?tmpLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; geofla:numInsee ?insee.
    }
}
"""

# Détection des doublons sur les codes postaux

query2 = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geofla: <http://data.ign.fr/def/geofla#>
prefix sys: <http://www.ontotext.com/owlim/system#>
PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>

INSERT {
    GRAPH ?g {
        ?landmark skos:exactMatch ?tmpLandmark.
    }
}
WHERE
{
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?g)
    {
        SELECT DISTINCT ?postalCode {
            GRAPH ?g {
                ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
            }
        }
    }
    BIND(URI(CONCAT(STR(URI(ban:)), "LM_", STRUUID())) AS ?landmark)

    GRAPH ?g {
        ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:PostalCode; rdfs:label ?postalCode.
    }
}
"""

# Détection des doublons sur les voies (même nom et appartient à la même commune ou au même arrondissement)

query3 = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geofla: <http://data.ign.fr/def/geofla#>
prefix sys: <http://www.ontotext.com/owlim/system#>
PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>

INSERT {
    GRAPH ?g {
        ?landmark skos:exactMatch ?tmpLandmark.
    }
}
WHERE
{
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?g)
    {
        SELECT DISTINCT ?label ?district WHERE {
            GRAPH ?g {
                ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; rdfs:label ?label.
                ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                ?district skos:exactMatch ?tmpDistrict.
            }
        }
    }
    BIND(URI(CONCAT(STR(URI(ban:)), "LM_", STRUUID())) AS ?landmark)

    GRAPH ?g {
        ?tmpLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; rdfs:label ?label.
                ?addrSeg a addr:AddressSegment; addr:relatum ?tmpLandmark; addr:nextStep [a addr:AddressSegment; addr:relatum ?tmpDistrict].
                ?tmpDistrict a addr:Landmark; addr:isLandmarkType ltype:District.
                ?district skos:exactMatch ?tmpDistrict.
    }
}
"""

# Transfert des données des ressources temporaires vers les permanentes.

query4 = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geofla: <http://data.ign.fr/def/geofla#>
PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>

DELETE {
    GRAPH ?g {
        ?s ?p ?tmpLandmark.
        ?tmpLandmark ?p ?o.
    }
}
INSERT {
    GRAPH ?g {
        ?s ?p ?landmark.
        ?landmark ?p ?o.
    }
}
WHERE {
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?g)
    GRAPH ?g {
        ?landmark skos:exactMatch ?tmpLandmark.
        {?tmpLandmark ?p ?o} UNION {?s ?p ?tmpLandmark}
        FILTER (?p != skos:exactMatch)
    }
} ; 

DELETE {
    ?landmark skos:exactMatch ?tmpLandmark.
}
WHERE {
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?g)
    GRAPH ?g {
        ?landmark skos:exactMatch ?tmpLandmark.
    }
}
"""

queries = [query1, query2, query3, query4]
for query in queries:
    gd.update_query(query, graphdb_url, project_name)

### Ajout d'événéments / de changements ainsi que des attributs à tous les repères

In [ ]:
query = f"""
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geofla: <http://data.ign.fr/def/geofla#>
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
PREFIX ban: <http://rdf.geohistoricaldata.org/id/address/sources/ban/>
PREFIX ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/>
PREFIX atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/>
DELETE {{
    GRAPH ?g {{ 
        ?landmark geo:asWKT ?geom; geofla:numInsee ?inseeCode.
    }}
}}
INSERT {{
    GRAPH <{ban_graph_uri}> {{
        ?landmark addr:hasAttribute ?nameAttribute, ?geomAttribute.
        ?nameAttribute a addr:Attribute; addr:isAttributeType atype:NameAttribute; addr:version ?versionNameAttribute1, ?versionNameAttribute2.
        ?geomAttribute a addr:Attribute; addr:isAttributeType atype:GeometryAttribute; addr:version ?versionGeomAttribute.
        ?versionNameAttribute1 a addr:AttributeVersion; addr:versionValue ?label.
        ?versionNameAttribute2 a addr:AttributeVersion; addr:versionValue ?inseeCode.
        ?versionGeomAttribute a addr:AttributeVersion; addr:versionValue ?geom.
    }}  
}}
WHERE {{
    GRAPH <{ban_graph_uri}> {{
        {{
            SELECT * {{
                ?landmark a addr:Landmark; rdfs:label ?label.
                OPTIONAL {{?landmark geo:asWKT ?geom}}
                OPTIONAL {{?landmark geofla:numInsee ?inseeCode}}
            }}
        }}
        BIND(URI(CONCAT(STR(URI(ban:)), "AN_", STRUUID())) AS ?nameAttribute)
        BIND(URI(CONCAT(STR(URI(ban:)), "ANV_", STRUUID())) AS ?versionNameAttribute1)
        BIND(IF(BOUND(?inseeCode), URI(CONCAT(STR(URI(ban:)), "ANV_", STRUUID())), ?x) AS ?versionNameAttribute2)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(ban:)), "AG_", STRUUID())), ?x) AS ?geomAttribute)
        BIND(IF(BOUND(?geom), URI(CONCAT(STR(URI(ban:)), "AGV_", STRUUID())), ?x) AS ?versionGeomAttribute)
    }}
}}
"""

gd.update_query(query, graphdb_url, project_name)

# Création des faits

## Variables liées au graphe des faits

In [ ]:
# Définition du graphe nommé
facts_graph_name = "facts"
facts_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, facts_graph_name)

# Définition du graphe nommé
source_facts_graph_name = "source_facts"
source_facts_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, source_facts_graph_name)

## Import des données de la ville de Paris comme étant des faits pour les voies

In [ ]:
query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#> 
PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>

INSERT {{
    GRAPH <{facts_graph_uri}> {{
        ?landmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; rdfs:label ?label.

    }}
    GRAPH <{source_facts_graph_uri}> {{
        ?landmark addr:isCreatedBy ?sourceLandmark.

    }}
}}
WHERE {{
    GRAPH <{vpt_graph_uri}> {{
        {{
            SELECT * {{
                ?sourceLandmark a addr:Landmark; rdfs:label ?label.
            }}
        }}
        BIND(URI(CONCAT(STR(URI(facts:)), "LM_", STRUUID())) AS ?landmark)
    }}
}}
"""

gd.update_query(query, graphdb_url, project_name)

## Liage des voies présentes dans les sources (BAN et Wikidata) avec celles présentes dans le graphe des faits

In [ ]:
# Définition de graphes nommés temporaires
tmp_graph_name = "tmp"
tmp_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, tmp_graph_name)

label_var = "?label"
norm_label_var = "?normLabel"
norm_label_function = sp.get_lower_simplified_french_street_name_function(label_var)

# Trois étapes :
# - créer un label normalisé pour les voies du graphe des faits dans un graphe temporaire ;
# - créer un label normalisé pour les voies du graphe de la BAN et de Wikidata dans ce même graphe temporaire ;
# - création d'un lien entre les voies ayant un label normalisé similaire dans le graphe des faits.

query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>

INSERT {{
    GRAPH <{tmp_graph_uri}> {{
        ?s skos:hiddenLabel {norm_label_var}.
    }}
}}
WHERE {{
    ?s a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare; rdfs:label {label_var}.
    BIND({norm_label_function} AS {norm_label_var})
}} ;

INSERT {{
    GRAPH <{source_facts_graph_uri}> {{
        ?factsLandmark addr:isReferencedBy ?sourceLandmark.
    }}
}}
WHERE {{
    FILTER (?g in (<{ban_graph_uri}>, <{wdpt_graph_uri}>))
    
    GRAPH ?g {{
        ?sourceLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
    }}

    GRAPH <{facts_graph_uri}> {{
        ?factsLandmark a addr:Landmark; addr:isLandmarkType ltype:Thoroughfare.
    }}
    GRAPH <{tmp_graph_uri}> {{
        ?sourceLandmark skos:hiddenLabel ?label.
        ?factsLandmark skos:hiddenLabel ?label.
    }}
}}
"""
print(query)
gd.update_query(query, graphdb_url, project_name)

## Import des données de la BAN comme étant des faits pour les adresses

### Lier les voies présentes dans le graphe des faits

### Insertion des données des adresses dans le graphe des faits

In [ ]:
# Créer des ressources de type Address dans le graphe des faits et créer un lien de provenance
query1 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
INSERT {
    GRAPH ?gf {
        ?factAddress a addr:Address; rdfs:label ?label.
    }
    GRAPH ?gsf {
        ?factAddress addr:isCreatedBy ?address.
    }
}
WHERE {
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?gs)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/facts> AS ?gf)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/source_facts> AS ?gsf)
    {
        SELECT * {  
            GRAPH ?gs {
                ?address a addr:Address.
                OPTIONAL {?address rdfs:label ?label}
            }
        }
    }
    BIND(URI(CONCAT(STR(URI(facts:)), "ADDR_", STRUUID()))AS ?factAddress)
}
"""

# Créer des ressources de type AddressSegment dans le graphe des faits et créer un lien de provenance
query2 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
INSERT {
    GRAPH ?gf {
        ?factAddrSeg a ?addrSegClass; addr:isLandmarkRelationType ?landmarkRelationType.
    }
    GRAPH ?gsf {
        ?factAddrSeg addr:isCreatedBy ?addrSeg.
    }
}
WHERE {
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?gs)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/facts> AS ?gf)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/source_facts> AS ?gsf)
    {
        SELECT * {  
            GRAPH ?gs {
                {?addrSeg a addr:AddressSegment.
                    MINUS{?addrSeg a addr:FinalAddressSegment}
                    BIND(addr:AddressSegment AS ?addrSegClass)
                }UNION{
                    ?addrSeg a addr:FinalAddressSegment.
                    BIND(addr:FinalAddressSegment AS ?addrSegClass)
                }
                ?addrSeg addr:isSpatialRelationType ?spatialRelationType
            }
        }
    }
    BIND(URI(CONCAT(STR(URI(facts:)), "AS_", STRUUID()))AS ?factAddrSeg)
}
"""

# Création des repères qui n'existent pas dans le graphe des faits (repères de la BAN qui sont reliés à aucun repère du graphe des faits)
query3 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX facts: <http://rdf.geohistoricaldata.org/id/address/facts/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
INSERT {
    GRAPH ?gf {
       ?factsLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; rdfs:label ?label. 
    }
    GRAPH ?gsf {
        ?factsLandmark addr:isCreatedBy ?sourceLandmark.
    }
}
WHERE {
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?gs)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/facts> AS ?gf)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/source_facts> AS ?gsf)

    GRAPH ?gs {
        ?sourceLandmark a addr:Landmark; addr:isLandmarkType ?landmarkType; rdfs:label ?label.
    }
    MINUS {?l addr:isReferencedBy ?sourceLandmark.}

 	BIND(URI(CONCAT(STR(URI(facts:)), "LM_", STRUUID())) AS ?factsLandmark)
}
"""

# Propagation des liens (firstStep, nextStep, targets, locatum, relatum) entre éléments de la BAN (Address, AddresSegment, Landmark) 
query4 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
INSERT {
    GRAPH ?gf {
        ?factsElem1 ?p ?factsElem2.
    }
}
WHERE {
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/ban> AS ?gs)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/facts> AS ?gf)
    BIND(<http://localhost:7200/repositories/addresses_from_factoids/rdf-graphs/source_facts> AS ?gsf)

    GRAPH ?gs {
        ?sourceElem1 ?p ?sourceElem2.
    }
    GRAPH ?gsf {
        ?factsElem1 addr:isReferencedBy ?sourceElem1.
        ?factsElem2 addr:isReferencedBy ?sourceElem2.
    }
}
"""

queries = [query1, query2, query3, query4]
for query in queries:
    gd.update_query(query, graphdb_url, project_name)